# Earthquake Data Collection

The following code was used to collect the earthquake data from the [USGS Earthquake API](https://earthquake.usgs.gov/fdsnws/event/1/):

In [1]:
import datetime as dt
import itertools
import requests

api = 'https://earthquake.usgs.gov/fdsnws/event/1/{}'
payload = {
    'format': 'geojson',
    'starttime': dt.datetime(2020, 1, 1),
    'endtime': dt.datetime(2021, 1, 1),
    'eventtype': 'earthquake'
}

response = requests.get(api.format('count'), params=payload)

if response.ok:
    count = response.json()['count']
    offset = 1
    earthquakes = []
    request_count = itertools.count(1)
    while offset < count:
        response = requests.get(
            api.format('query'), 
            params={
                **payload, **{
                    'limit': 20_000,
                    'orderby': 'time-asc',
                    'offset': offset
                }
            }
        )
        if not response.ok:
            print(f'Request failed with status {response.status_code}')
            break
        results = response.json()
        earthquakes.extend(results['features'])
        offset += results['metadata']['count']

        print(
            f'\rRequest {next(request_count)} complete. Progress: {(offset - 1)/count:.2%}',
            end=''
        )

Request 10 complete. Progress: 100.00%

## Generating the `earthquakes.geojson` file

The GeoJSON format provides the `geometry` field, which we will use with GeoViews in the workshop. A single record from the API looks like this:

In [2]:
earthquakes[0]

{'type': 'Feature',
 'properties': {'mag': 2.75,
  'place': '80 km N of Isabela, Puerto Rico',
  'time': 1577836916590,
  'updated': 1577988904335,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/pr2020001021',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=pr2020001021&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'reviewed',
  'tsunami': 0,
  'sig': 116,
  'net': 'pr',
  'code': '2020001021',
  'ids': ',pr2020001021,',
  'sources': ',pr,',
  'types': ',origin,phase-data,',
  'nst': 3,
  'dmin': 0.7502,
  'rms': 0.45,
  'gap': 358,
  'magType': 'md',
  'type': 'earthquake',
  'title': 'M 2.8 - 80 km N of Isabela, Puerto Rico'},
 'geometry': {'type': 'Point', 'coordinates': [-67.1275, 19.2175, 12]},
 'id': 'pr2020001021'}

Using GeoPandas, we can work with this like a pandas DataFrame:

In [3]:
import geopandas as gpd

gdf = gpd.GeoDataFrame.from_features(
    earthquakes, columns=['geometry', 'mag', 'place', 'time', 'tsunami', 'magType']
)
gdf.head()

,geometry,mag,place,time,tsunami,magType
0,POINT Z (-67.12750 19.21750 12.00000),2.75,"80 km N of Isabela, Puerto Rico",1577836916590,0,md
1,POINT Z (-67.09010 19.07660 6.00000),2.55,"64 km N of Isabela, Puerto Rico",1577837018210,0,md
2,POINT Z (-66.85410 17.87050 6.00000),1.81,"12 km SSE of Maria Antonia, Puerto Rico",1577837109440,0,md
3,POINT Z (-66.86360 17.89930 8.00000),1.84,"9 km SSE of Maria Antonia, Puerto Rico",1577837136930,0,md
4,POINT Z (-66.86850 17.90660 8.00000),1.64,"8 km SSE of Maria Antonia, Puerto Rico",1577837360060,0,md


**Warning**: The earthquake data is frequently updated (even for past events). If you run this cell, your results may be different.

---

Save to a GeoJSON file for use later:

In [4]:
gdf.to_file('../earthquakes.geojson', driver='GeoJSON')